In [4]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import rasterio
import os
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import xarray
import itertools
import matplotlib.colors as mcolors
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature
import warnings
from shapely.geometry import box


In [53]:
# just find and open the species files according to the habitats resp. forest vs non-forest
# Define the netcdf path format
netcdf_path_format = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_{}.nc"

taxas=["Mammals","Amphibians"]
model_name = 'GFDL-ESM2M'
model = "GAM"
time = [65]  # Replace with the appropriate time value
scenario="rcp26"

# Load the CSV files containing species names for forest and non-forest categories
csv_file_path_nonforest = f'/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_non_forest{time}.csv'
species_df_nonforest = pd.read_csv(csv_file_path_nonforest)

csv_file_path_forest = f'/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_forest{time}.csv'
species_df_forest = pd.read_csv(csv_file_path_forest)


# Function to open netcdf files for a given species name in the forest category
def open_forest(taxa,species_name_forest):
    netcdf_path = netcdf_path_format.format(model, taxa, model_name, scenario, species_name_forest, time)
    ds_forest = xr.open_dataset(netcdf_path, decode_times=False)
    # Perform analysis or processing on the dataset

# Function to open netcdf files for a given species name in the non-forest category
def open_nonforest(taxa,species_name_nonforest):
    netcdf_path = netcdf_path_format.format(model, taxa, model_name, scenario, species_name_nonforest, time)
    ds_nonforest = xr.open_dataset(netcdf_path, decode_times=False)
    

for species_name in species_df_nonforest['Species']:
    found = False
    for taxa in taxas:
        try:
            open_nonforest(taxa, species_name)
            found = True
            break  # Exit the loop if species is found in any taxa
        except:
            continue
    if found:
        print(f"Species {species_name} found in {taxa}.")
        
for species_name in species_df_forest['Species']:
    species_name_cleaned = species_name.strip()  # Remove leading and trailing spaces
    found = False
    for taxa in taxas:
        try:
            open_forest(taxa, species_name_cleaned)
            found = True
            break  # Exit the loop if species is found in any taxa
        except:
            continue
    if found:
        print(f"Species {species_name_cleaned} found in {taxa}.")


Species Sorex_camtschatica found in Mammals.
Species Axis_porcinus found in Mammals.
Species Phyllotis_amicus found in Mammals.
Species Crinia_bilingua found in Amphibians.
Species Rhinolophus_euryotis found in Mammals.
Species Hypsugo_kitcheneri found in Mammals.
Species Bdeogale_nigripes found in Mammals.
Species Hystrix_javanica found in Mammals.
Species Spilogale_putorius found in Mammals.
Species Cheirogaleus_crossleyi found in Mammals.
Species Hydrochoerus_hydrochaeris found in Mammals.
Species Hyloscirtus_palmeri found in Amphibians.
Species Rhinella_dapsilis found in Amphibians.
Species Smilisca_phaeota found in Amphibians.
Species Scinax_cardosoi found in Amphibians.
Species Amolops_larutensis found in Amphibians.
Species Allobates_talamancae found in Amphibians.
Species Litoria_peronii found in Amphibians.
Species Boophis_andohahela found in Amphibians.
Species Dendropsophus_seniculus found in Amphibians.


In [6]:
# just find and open the species files according to the habitats resp. forest vs non-forest
# Define the netcdf path format
netcdf_path_format = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_{}.nc"

taxas=["Mammals","Amphibians"]
model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
model = "GAM"
time = [65]  # Replace with the appropriate time value
scenarios=["rcp26"]

# Load the CSV files containing species names for forest and non-forest categories
csv_file_path_nonforest = f'/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_non_forest{time}.csv'
species_df_nonforest = pd.read_csv(csv_file_path_nonforest)

csv_file_path_forest = f'/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_forest{time}.csv'
species_df_forest = pd.read_csv(csv_file_path_forest)

newvalue_dict = {model_name: {} for model_name in model_names}
sum_bin_dict_forest = {model_name: {} for model_name in model_names}

# Function to open netcdf files for a given species name in the forest category
def open_forest(taxa,species_name_forest):
    for model_name in model_names:
        for scenario in scenarios:
            netcdf_path = netcdf_path_format.format(model, taxa, model_name, scenario, species_name_forest, time)
            ds_forest = xr.open_dataset(netcdf_path, decode_times=False)
            sum_bin = ds_forest["sum_bin"]
            sum_bin = (sum_bin > 0.00)
            
            sum_bin_dict_forest[model_name][species_name_forest] = sum_bin
            
            projection_dict= {}
            
        for species_name in species_name_forest:
            sum_bin_list=[] 
            for model_name in model_names:
                sum_bin = sum_bin_dict_forest[model_name][species_name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = sum_bin_concat.mean(dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_sum_bin
            
            mean_sum_bin_list = list(projections_dict.values())
            mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").mean(dim="species")  # Ensemble mean over species
            mean_sum_bin_forest = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin_forest

    # Perform analysis or processing on the dataset

# Function to open netcdf files for a given species name in the non-forest category
def open_nonforest(taxa,species_name_nonforest):
    netcdf_path = netcdf_path_format.format(model, taxa, model_name, scenario, species_name_nonforest, time)
    ds_nonforest = xr.open_dataset(netcdf_path, decode_times=False)
    

for species_name in species_df_nonforest['Species']:
    found = False
    for taxa in taxas:
        try:
            open_nonforest(taxa, species_name)
            found = True
            break  # Exit the loop if species is found in any taxa
        except:
            continue
    if found:
        print(f"Species {species_name} found in {taxa}.")
        
for species_name in species_df_forest['Species']:
    species_name_cleaned = species_name.strip()  # Remove leading and trailing spaces
    found = False
    for taxa in taxas:
        try:
            open_forest(taxa, species_name_cleaned)
            found = True
            break  # Exit the loop if species is found in any taxa
        except:
            continue
    if found:
        print(f"Species {species_name_cleaned} found in {taxa}.")
